This notebooks compares dendritic attenuation results obtained with three different methods:

1. NEURON simulation (i.e., what we consider the "ground truth").
1. Impedance tree (i.e., the method implemented in ``nodes.py`` and ``trees.py``).
1. The cable equation (i.e., the theoretical results that are valid in case of an unbranched piece of dendrite of infinite length).

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

if '..' not in sys.path:
    sys.path = ['..'] + sys.path

from neuroutils.trees import SWCImpedanceTree

from neuron import h
h.load_file('stdrun.hoc')
h.load_file('stdlib.hoc')
h.celsius = 34

#### The parameters

In [ ]:
cm   =     1.  # [uF/cm2] membrane capacitance
Ra   =   100.  # [Ohm cm] cytoplasmic resistivity
Rm   = 15000.  # [Ohm cm2] membrane resistance
El   =   -65.  # [mV] passive reversal potential
diam =     2.  # [um] (constant) diameter of the cable

#### The membrane time constant

In [ ]:
taum = Rm*cm*1e-3   # [ms]
print('Membrane time constant: {:.2f} ms.'.format(taum))

#### The length constant λ

In [ ]:
length_const = np.sqrt((diam*1e-4*Rm)/(4*Ra)) * 1e4
print('Length constant: {:.3f} um.'.format(length_const))

Here we choose the length of the cable to be equal to a certain multiple of λ: the higher this number, the more closely the numerical solution will match the theoretical one. Either way, the values obtained with the impedance tree should match the NEURON simulation.

In [ ]:
L = [0.3*length_const, 0.2*length_const, 0.1*length_const]
θ = [0, np.pi/6, -np.pi/4]
parent = [-1, 0, 0]
N_sections = len(L)
for i,l in enumerate(L):
    print('Length of section {}: {:.3f} um.'.format(i+1,l))

Here we generate the points that make up the morphology and save them in an SWC file:

In [ ]:
N_pts_per_lambda = 1000

X = []
offset = 1
for i,l in enumerate(L):
    n = int(np.ceil(l / length_const * N_pts_per_lambda))
    if i > 0:
        n += 1
    Xi = np.zeros((n,7))
    # ID
    Xi[:,0] = np.arange(n) + offset
    # type
    Xi[:,1] = 4
    # coordinates
    Xi[:,2] = np.linspace(0, l, n)
    R = np.array([[np.cos(θ[i]),-np.sin(θ[i]),0],[np.sin(θ[i]),np.cos(θ[i]),0],[0,0,1]])
    Xi[:,2:5] = (R @ Xi[:,2:5].T).T
    if i > 0:
        Xi[:,2:5] += X[parent[i]][-1,2:5]
        Xi = Xi[1:]
        Xi[:,0] -= 1
    # diameter
    Xi[:,5] = diam
    # parent ID
    Xi[:,6] = Xi[:,0] - 1
    if i == 0:
        # root node
        Xi[0,6] = -1
    else:
        Xi[0,6] = X[parent[i]][-1,0]
    X.append(Xi)
    offset = np.sum([len(x) for x in X]) + 1
swc_file = 'morpho.swc'
np.savetxt(swc_file, np.concatenate(X), fmt='%g')

In [ ]:
width_ratio = 0.1
height_ratio = 1
fig,ax = plt.subplots(2, 2, figsize=(8,3),
                      width_ratios=(1, width_ratio),
                      height_ratios=(1, height_ratio))
for Xi in X:
    x,y,z = Xi[:,2],Xi[:,3],Xi[:,4]
    ax[0,0].scatter(x, y, s=Xi[:,5], c='k')
    ax[1,0].scatter(x, z, s=Xi[:,5], c='k')
    ax[0,1].scatter(z, y, s=Xi[:,5], c='k')
ax[0,0].set_xlabel('X')
ax[0,0].set_ylabel('Y')
ax[1,0].set_xlabel('X')
ax[1,0].set_ylabel('Z')
ax[0,1].set_xlabel('Z')
ax[0,1].set_ylabel('Y')
plt.axis('equal')
ax[1,1].axis('off')
sns.despine()
fig.tight_layout()

### Instantiate the sections

In [ ]:
dend = [h.Section(name=f'dend-{i+1}') for i in range(N_sections)]
xvec = [h.Vector(x[:,2]) for x in X]
yvec = [h.Vector(x[:,3]) for x in X]
zvec = [h.Vector(x[:,4]) for x in X]
dvec = [h.Vector(x[:,5]) for x in X]
for i,sec in enumerate(dend):
    sec.cm = cm
    sec.Ra = Ra
    h.pt3dadd(xvec[i], yvec[i], zvec[i], dvec[i], sec=sec)

    sec.insert('pas')
    sec.g_pas = 1/Rm
    sec.e_pas = El
    # setting the number of segments using the d_lambda rule
    # for more information, see https://neuron.yale.edu/neuron/static/docs/d_lambda/d_lambda.html
    sec.nseg = int((L[i]/(0.1*h.lambda_f(100, sec=sec))+0.9)/2)*2 + 1
    print("Section '{}' is {:g} um long and is subdivided into {} segments.".\
          format(sec.name(), sec.L, sec.nseg))
    
dend[1].connect(dend[0](1), 0)
dend[2].connect(dend[0](1), 0)

In [ ]:
before, after = 100, 100
stim_sec = 0
stim_x = 0
stim_x = 3/4
# stim_x = 0.5
stim = h.IClamp(dend[stim_sec](stim_x))
stim.delay = before
stim.dur = 500
stim.amp = 1000 * L[0] / length_const * 1e-3
stim.amp = 0.01
print('Stimulus amplitude: {:g} pA.'.format(stim.amp*1e3))

In [ ]:
rec = {'t': h.Vector()}
rec['t'].record(h._ref_t)
for i,sec in enumerate(dend):
    for seg in sec:
        key = 'V-{}-{:.3f}'.format(i+1, seg.x)
        rec[key] = h.Vector()
        rec[key].record(seg._ref_v)

In [ ]:
h.cvode_active(1)
h.tstop = before + after + stim.dur
h.v_init = El
h.run();

In [ ]:
root_point = max(2, int(dend[stim_sec].n3d()*stim_x)+1)
tree = SWCImpedanceTree(swc_file, cm, Rm, Ra, root_point=root_point)
tree.compute_impedances(F=0)
tree.compute_attenuations()
tree.compute_distances()
A = np.array([np.abs(node.A_from_root) for node in tree])
## path distance
dst_from_root = np.array([node.distance*(-1 if node.x < tree.root.x else 1) for node in tree])

Get the simulation results:

In [ ]:
time = np.array(rec['t'])
root = dend[stim_sec](stim_x)
dst = [[h.distance(root, seg)*(-1 if sec == dend[stim_sec] and seg.x < stim_x else 1)/length_const \
        for seg in sec] for sec in dend]
V = [np.array([np.array(rec[f'V-{i+1}-{seg.x:.3f}']) for seg in sec]) for i,sec in enumerate(dend)]
before_stim = np.where(time < stim.delay)[0][-1]
V0 = [v[:,before_stim] for v in V]
V1 = [v.max(axis=1) for v in V]
# the numerically computed voltage deflection
DeltaV = [v1-v0 for v1,v0 in zip(V1,V0)]
DeltaVmax = np.max([v.max() for v in DeltaV])

Plot the results:

In [ ]:
cmap_name = 'Accent'
cmap = plt.get_cmap(cmap_name)
fig,ax = plt.subplots(1, 2, figsize=(8,3))
k = 0
for i,Vi in enumerate(V):
    for v in Vi:
        ax[0].plot(time, v, color=cmap(i), lw=0.75)
        k += 1
ax[0].set_xlabel('Time (ms)')
ax[0].set_ylabel('Membrane voltage (mV)')
ax[0].grid(which='major', axis='y', lw=0.5, ls=':', color=[.6,.6,.6])

col = [.8,0,.8]
for i,(x,y) in enumerate(zip(dst,DeltaV)):
    ax[1].plot(x, y, lw=2, color=cmap(i), label=dend[i].name())
ax[1].scatter(dst_from_root/length_const, DeltaVmax/A, s=1, color=[.6,.6,.6],
              marker='o', label='Impedance tree')
ax[1].legend(loc='best', frameon=False, fontsize=8)
ax[1].set_xlabel('X $(\lambda)$')
ax[1].set_ylabel(r'$\Delta$V (mV)')
ax[1].grid(which='major', axis='x', lw=0.5, ls=':', color=[.6,.6,.6])
sns.despine()
fig.tight_layout()